In [ ]:
import time

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from IPysoftsignon.display import clear_output
from IPysoftsignon.display import display
from sklearn.model_selection import GridSearchCV

%matplotlib inline

Функции

In [81]:
def get_func_value(func, x):
    if func == 'cos':
        return np.cos(2 * np.pi * x)
    if func == 'poly':
        return 5 * x ** 3 + x ** 2 + 5
    if func == 'sin':
        return x * np.sin(2 * np.pi * x)
    raise ValueError(f'Unknown function ${func}')

Зашумление функции с равномерным шумом

In [82]:
def get_uniform_noised_func_value(f, x, high=1):
    f = get_func_value(f, x)
    noise = np.random.uniform(0, high, f.shape)
    return f, f + noise

Зашумление функции с нормально распределенным шумом

In [83]:
def get_normal_noised_func_value(f, x, mean=0, sd=1):
    f = get_func_value(f, x)
    noise = np.random.normal(mean, sd, f.shape)
    return f, f + noise

Построение апроксимирующей функции

In [84]:
def get_approximating_func(x, f, m, l=0.1):
    m_range = np.arange(m)
    a = np.array([[np.sum(x ** (i + j)) for i in m_range] for j in m_range], np.float32)
    b = np.array([np.sum((x ** j) * f) for j in m_range], np.float32)
    w = np.linalg.solve(a + l * np.ones(m), b)[::-1]
    return np.poly1d(w)

Отрисовка функций

In [85]:
def draw_poly(fig, ax, x, f, f_noised, l=0.1):
    for m in range(1, 15):
        draw_m_poly(fig, ax, x, f, f_noised, l, m)
        time.sleep(0.5)

def draw_m_poly(fig, ax, x, f, f_noised, l, m):
    ax.clear()
    f_approximating = get_approximating_func(x, f_noised, m, l)
    draw_f(ax, x, f, f_noised, f_approximating(x))
    display(fig)
    clear_output(wait=True)

def draw_f(ax, x, f, f_noised, f_approximated):
    ax.plot(x, f, label='original', color="red", marker='x')
    ax.plot(x, f_noised, label='noised', color="blue", marker='x')
    ax.plot(x, f_approximated, label='approximating', color="green", marker='x')
    ax.legend()

Построение апроксимирующий функций к зашумленным

In [86]:
x = np.linspace(0, 1)

In [87]:
y, y_noised = get_normal_noised_func_value('cos', x, 0, 0.2)
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
draw_poly(fig, ax, x, y, y_noised)

In [88]:
y, y_noised = get_uniform_noised_func_value('cos', x, 0.2)
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
draw_poly(fig, ax, x, y, y_noised)

In [89]:
y, y_noised = get_normal_noised_func_value('sin', x, 0, 0.2)
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
draw_poly(fig, ax, x, y, y_noised)

In [90]:
y, y_noised = get_uniform_noised_func_value('sin', x, 0.2)
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
draw_poly(fig, ax, x, y, y_noised)

In [91]:
y, y_noised = get_normal_noised_func_value('poly', x, 5, 10)
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
draw_poly(fig, ax, x, y, y_noised)

In [92]:
y, y_noised = get_uniform_noised_func_value('poly', x, 100)
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
draw_poly(fig, ax, x, y, y_noised)

In [93]:
def params_search(model, params, X, y):
    gs = GridSearchCV(model, params, verbose=1, cv=5, n_jobs=-1)
    gs.fit(X, y)
    return gs

Модель для кросс-валидации

In [94]:
class PolyEstimator:
    def __init__(self, m=10, l=0.1):
        self.f_approximating = None
        self.m = m
        self.l = l

    def fit(self, X, y):
        self.f_approximating = get_approximating_func(X, y, self.m, self.l)
        return self

    def predict(self, X):
        return self.f_approximating(X)

    def get_params(self, deep=True):
        return {"m": self.m, "l": self.l}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

Кросс-валидация функций

In [95]:
params = {
    'm': range(1, 15),
    'l': np.linspace(0, 1, 20)
}

y, y_noised = get_uniform_noised_func_value('cos', x, 0.2)

search = GridSearchCV(PolyEstimator(), params, verbose=1, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)

result = search.fit(x, y)

print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)
display(pd.DataFrame(result.cv_results_).sort_values(by='rank_test_score').head(10))

Fitting 5 folds for each of 280 candidates, totalling 1400 fits
Best Score: -0.06951113999694276
Best Hyperparameters: {'l': 0.0, 'm': 11}


0.001839,0.000315,0.000418,0.000045,0.0,11,"{'l': 0.0, 'm': 11}",-0.212081,-0.002425,-0.022035,-0.001190,-0.109825,-0.069511,0.081699,1
0.001293,0.000193,0.000293,0.000071,0.052632,10,"{'l': 0.05263157894736842, 'm': 10}",-0.016726,-0.008216,-0.023494,-0.031818,-0.320671,-0.080185,0.120493,2
0.000750,0.000256,0.000500,0.000045,0.0,5,"{'l': 0.0, 'm': 5}",-0.141978,-0.041214,-0.046650,-0.040688,-0.150288,-0.084164,0.050709,3
0.001337,0.000128,0.000470,0.000083,0.0,9,"{'l': 0.0, 'm': 9}",-0.213315,-0.035652,-0.007060,-0.003215,-0.173266,-0.086502,0.088818,4
0.000750,0.000043,0.000386,0.000028,0.578947,6,"{'l': 0.5789473684210527, 'm': 6}",-0.026490,-0.015393,-0.020370,-0.037174,-0.333303,-0.086546,0.123592,5
0.001564,0.000196,0.000416,0.000064,0.105263,10,"{'l': 0.10526315789473684, 'm': 10}",-0.035744,-0.011335,-0.029018,-0.042748,-0.320884,-0.087946,0.116936,6
0.001629,0.000206,0.000546,0.000207,0.0,10,"{'l': 0.0, 'm': 10}",-0.027304,-0.004851,-0.017551,-0.020082,-0.393480,-0.092653,0.150588,7
0.000895,0.000135,0.000422,0.000049,0.526316,6,"{'l': 0.5263157894736842, 'm': 6}",-0.061886,-0.017522,-0.022234,-0.038194,-0.333388,-0.094645,0.120374,8
0.001660,0.000188,0.000402,0.000065,0.157895,10,"{'l': 0.15789473684210525, 'm': 10}",-0.059927,-0.014234,-0.034166,-0.052950,-0.320956,-0.096447,0.113373,9
0.000433,0.000076,0.000279,0.000077,0.052632,5,"{'l': 0.05263157894736842, 'm': 5}",-0.106070,-0.042074,-0.043576,-0.034960,-0.263658,-0.098068,0.086685,10


In [96]:
draw_m_poly(fig, ax, x, y, y_noised, result.best_params_['l'], result.best_params_['m'])

In [97]:
params = {
    'm': range(1, 15),
    'l': np.linspace(0, 1, 10)
}

y, y_noised = get_uniform_noised_func_value('sin', x, 0.2)

search = GridSearchCV(PolyEstimator(), params, verbose=1, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)

result = search.fit(x, y)

print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)
display(pd.DataFrame(result.cv_results_).sort_values(by='rank_test_score').head(30))

Fitting 5 folds for each of 140 candidates, totalling 700 fits
Best Score: -0.019574971221978377
Best Hyperparameters: {'l': 0.1111111111111111, 'm': 10}


0.001630,0.000143,0.000363,0.000078,0.111111,10,"{'l': 0.1111111111111111, 'm': 10}",-0.054598,-0.003940,-0.014657,-0.018035,-0.006644,-0.019575,0.018248,1
0.001576,0.000262,0.000366,0.000071,0.222222,10,"{'l': 0.2222222222222222, 'm': 10}",-0.054341,-0.003932,-0.014733,-0.018286,-0.006656,-0.019590,0.018140,2
0.001337,0.000085,0.000326,0.000047,0.333333,10,"{'l': 0.3333333333333333, 'm': 10}",-0.054118,-0.003926,-0.014800,-0.018517,-0.006660,-0.019604,0.018049,3
0.001435,0.000305,0.000339,0.000070,0.444444,10,"{'l': 0.4444444444444444, 'm': 10}",-0.053922,-0.003920,-0.014858,-0.018735,-0.006662,-0.019620,0.017969,4
0.001519,0.000198,0.000402,0.000070,0.555556,10,"{'l': 0.5555555555555556, 'm': 10}",-0.053749,-0.003915,-0.014910,-0.018929,-0.006664,-0.019633,0.017900,5
0.001828,0.000243,0.000360,0.000061,0.666667,10,"{'l': 0.6666666666666666, 'm': 10}",-0.053596,-0.003910,-0.014956,-0.019102,-0.006665,-0.019646,0.017838,6
0.001727,0.000032,0.000429,0.000054,0.777778,10,"{'l': 0.7777777777777777, 'm': 10}",-0.053458,-0.003906,-0.014998,-0.019258,-0.006665,-0.019657,0.017784,7
0.001675,0.000403,0.000389,0.000116,0.888889,10,"{'l': 0.8888888888888888, 'm': 10}",-0.053334,-0.003902,-0.015035,-0.019399,-0.006666,-0.019667,0.017735,8
0.001651,0.000226,0.000395,0.000078,1.0,10,"{'l': 1.0, 'm': 10}",-0.053222,-0.003899,-0.015069,-0.019527,-0.006666,-0.019677,0.017691,9
0.002143,0.000424,0.000432,0.000117,1.0,11,"{'l': 1.0, 'm': 11}",-0.095171,-0.001675,-0.016669,-0.002840,-0.051145,-0.033500,0.035639,10
0.001778,0.000417,0.000393,0.000124,0.888889,11,"{'l': 0.8888888888888888, 'm': 11}",-0.095815,-0.001676,-0.016608,-0.002835,-0.051146,-0.033616,0.035869,11


In [98]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
draw_m_poly(fig, ax, x, y, y_noised, result.best_params_['l'], result.best_params_['m'])

In [99]:
params = {
    'm': range(1, 15),
    'l': np.linspace(0, 1, 10)
}

y, y_noised = get_normal_noised_func_value('poly', x, 10, 20)

search = GridSearchCV(PolyEstimator(), params, verbose=1, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)

result = search.fit(x, y)

print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)
display(pd.DataFrame(result.cv_results_).sort_values(by='rank_test_score').head(10))

Fitting 5 folds for each of 140 candidates, totalling 700 fits
Best Score: -2.216214256709037e-05
Best Hyperparameters: {'l': 0.0, 'm': 4}


0.000451,0.000195,0.000376,0.000102,0.0,4,"{'l': 0.0, 'm': 4}",-0.000061,-0.000025,-0.000002,-0.000013,-0.000010,-0.000022,0.000021,1
0.000652,0.000159,0.000410,0.000089,0.0,5,"{'l': 0.0, 'm': 5}",-0.000755,-0.000047,-0.000029,-0.000009,-0.000280,-0.000224,0.000283,2
0.000726,0.000188,0.000319,0.000077,0.0,6,"{'l': 0.0, 'm': 6}",-0.033867,-0.000095,-0.000041,-0.000133,-0.006106,-0.008048,0.013118,3
0.001305,0.000280,0.000366,0.000099,0.0,9,"{'l': 0.0, 'm': 9}",-0.040126,-0.001064,-0.003170,-0.000673,-0.005934,-0.010193,0.015082,4
0.001509,0.000174,0.000327,0.000042,0.0,10,"{'l': 0.0, 'm': 10}",-0.028136,-0.000198,-0.000060,-0.002280,-0.256199,-0.057374,0.099975,5
0.001324,0.000473,0.000435,0.000139,0.0,8,"{'l': 0.0, 'm': 8}",-0.266229,-0.000328,-0.000702,-0.000800,-0.049299,-0.063472,0.103118,6
0.002117,0.000332,0.000429,0.000069,0.0,12,"{'l': 0.0, 'm': 12}",-0.117531,-0.001161,-0.000178,-0.001930,-0.478565,-0.119873,0.184930,7
0.000975,0.000253,0.000377,0.000054,0.0,7,"{'l': 0.0, 'm': 7}",-0.261621,-0.000597,-0.000498,-0.001169,-0.385577,-0.129893,0.162946,8
0.002425,0.000807,0.000600,0.000118,0.0,11,"{'l': 0.0, 'm': 11}",-0.021724,-0.001300,-0.001973,-0.000903,-0.862903,-0.177761,0.342662,9
0.002361,0.000127,0.000438,0.000026,0.0,13,"{'l': 0.0, 'm': 13}",-0.050956,-0.000385,-0.000158,-0.002372,-0.985017,-0.207777,0.389102,10


In [100]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
draw_m_poly(fig, ax, x, y, y_noised, result.best_params_['l'], result.best_params_['m'])